In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *

%matplotlib inline

In [4]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/cs230 project/collab/fer2013/fer2013.csv')

#print('Number of samples in the dataset: ', data.shape[0])
# Transform images from strings to lists of integers. TODO: use an array cast
data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
emotion_cat = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

# See the target distribution (check for imbalance)
#target_counts = data['emotion'].value_counts().reset_index(drop=False)
#target_counts.columns = ['emotion', 'number_samples']
#target_counts['emotion'] = target_counts['emotion'].map(emotion_cat)
#target_counts

# Select randomly 10 images
#random_seed = 1
#data_sample = data.sample(10, random_state=random_seed)
#f, axarr = plt.subplots(2, 5, figsize=(20, 10))

#i, j = 0, 0
#for idx, row in data_sample.iterrows():
#    img = np.array(row['pixels']).reshape(48,48)
#    axarr[i,j].imshow(img, cmap='gray')
#    axarr[i,j].set_title(emotion_cat[row['emotion']])
#    if j==4:
#        i += 1
#        j = 0
#    else:
#        j += 1

In [0]:
data_train = data[data['Usage']=='Training']
#print('Number samples in the training dataset: ', data_train.shape[0])

data_dev = data[data['Usage']=='PublicTest']
#print('Number samples in the development dataset: ', data_dev.shape[0])

# Retrieve train input and target
X_train, y_train = data_train['pixels'].tolist(), data_train['emotion'].values
# Reshape images to 4D (num_samples, width, height, num_channels)
X_train = np.array(X_train, dtype='float32').reshape(-1,48,48,1)
# Normalize images with max (the maximum pixel intensity is 255)
X_train = X_train/255.0

# Retrieve dev input and target
X_dev, y_dev = data_dev['pixels'].tolist(), data_dev['emotion'].values
X_dev = np.array(X_dev, dtype='float32').reshape(-1,48,48,1)
X_dev = X_dev/255.0

In [0]:
# Implement below paper CPCPCPFF depth 5, 2.4m params
# http://openaccess.thecvf.com/content_cvpr_2016_workshops/w28/papers/Kim_Fusing_Aligned_and_CVPR_2016_paper.pdf
# Reference: https://arxiv.org/pdf/1612.02903.pdf
model = tf.keras.models.Sequential(   
    [
      InputLayer(input_shape=(48,48,1),name="input"),
      Conv2D(filters=64,kernel_size=5,activation='relu',padding='same',name="conv1"),
      MaxPool2D(pool_size=(2,2),name="maxpool1"),     
      Conv2D(filters=96,kernel_size=5,activation='relu',padding='same',name="conv2"),
      MaxPool2D(pool_size=(2,2),name="maxpool2"),
      Conv2D(filters=256,kernel_size=5,activation='relu',padding='same',name="conv3"),
      MaxPool2D(pool_size=(2,2),name="maxpool3"),
      Conv2D(filters=256,kernel_size=5,activation='relu',padding='same',name="conv4"),
      Flatten(),
      Dense(2048,input_shape=(6*6*256,1),activation='relu',name='fc1'),
      Dropout(0.5),
      Dense(7,input_shape=(2048,1),activation='softmax',name='fc-softmax')
    ]
)

#TODO: weight decay of 0.0001...initial learning rate is set to 0.01 and reduced by a factor of 2 at every 25 epoch
sgd = tf.keras.optimizers.SGD(lr=0.01,momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train,batch_size=500,epochs=1000,validation_data=(X_dev, y_dev))

Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 6s 196us/sample - loss: 1.8211 - accuracy: 0.2500 - val_loss: 1.8024 - val_accuracy: 0.2502
Epoch 2/100
28709/28709 [==============================] - 5s 177us/sample - loss: 1.7707 - accuracy: 0.2702 - val_loss: 1.7184 - val_accuracy: 0.3162
Epoch 3/100
28709/28709 [==============================] - 5s 178us/sample - loss: 1.6818 - accuracy: 0.3377 - val_loss: 1.6244 - val_accuracy: 0.3661
Epoch 4/100
28709/28709 [==============================] - 5s 176us/sample - loss: 1.5864 - accuracy: 0.3888 - val_loss: 1.5362 - val_accuracy: 0.4115
Epoch 5/100
28709/28709 [==============================] - 5s 180us/sample - loss: 1.5078 - accuracy: 0.4211 - val_loss: 1.4952 - val_accuracy: 0.4199
Epoch 6/100
28709/28709 [==============================] - 5s 177us/sample - loss: 1.4491 - accuracy: 0.4446 - val_loss: 1.4330 - val_accuracy: 0.4550
Epoch 7/100
28709/28709 [====================